Assignment Question:

Utilize the tensorflow_privacy library to train a classification model on the MNIST dataset while employing the DPKerasSGDOptimizer as the training optimizer, ensuring that the training process is differentially private. Upon completion, calculate the epsilon value for the given training parameters.

Please take note of the following instructions:

*  Utilize the MNIST Dataset from Keras.datasets.
*  Implement a CNN architecture.
*  Suggested parameters for SGD Optimiser: batch_size = 250, l2_norm_clip = 1.5, noise_multiplier = 1.3, num_microbatches = 250, learning_rate = 0.25.
*  You can utilize the compute_dp_sgd_privacy_statement function to determine the epsilon value.


For reference, you can refer to the https://www.tensorflow.org/responsible_ai/privacy/guide.

In [ ]:
!pip install tensorflow-privacy

In [ ]:
import tensorflow_privacy
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
train, test = tf.keras.datasets.mnist.load_data()#load the dataset
#train test split
train_data, train_labels = train
test_data, test_labels = test
#normalise
train_data = np.array(train_data, dtype=np.float32) / 255
test_data = np.array(test_data, dtype=np.float32) / 255

train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

train_labels = np.array(train_labels, dtype=np.int32)
test_labels = np.array(test_labels, dtype=np.int32)

#one hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
#Defining parameters
epochs = 5
batch_size = 250
l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 250
learning_rate = 0.25

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [ ]:
#defining model using convolution and dense layer with softmax
model = tf.keras.Sequential([
    # Convolutional Layer 1
    tf.keras.layers.Conv2D(16, 8,
                           strides=2,
                           padding='same',
                           activation='relu',
                           input_shape=(28, 28, 1)),
    # Max Pooling Layer 1
    tf.keras.layers.MaxPool2D(2, 1),

    # Convolutional Layer 2
    tf.keras.layers.Conv2D(32, 4,
                           strides=2,
                           padding='valid',
                           activation='relu'),
    # Max Pooling Layer 2
    tf.keras.layers.MaxPool2D(2, 1),

    # Flatten Layer
    tf.keras.layers.Flatten(),

    # Dense Layer 1
    tf.keras.layers.Dense(32, activation='relu'),
    # Output Layer (Dense Layer 2)
    # Note: The number of units in the output layer should match the number of classes in your classification task.
    tf.keras.layers.Dense(10)  # Assuming 10 classes
])
#defining the optimiser with our parameters that we had defined earlier
optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)
#defining the loss function as cateorical cross entropy
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
#training the model
history=model.fit(train_data, train_labels,
          epochs=epochs,
          validation_data=(test_data, test_labels),
          batch_size=batch_size)

Epoch 1/5
240/240 [==============================] - 63s 256ms/step - loss: 0.5277 - accuracy: 0.8610 - val_loss: 0.4114 - val_accuracy: 0.8964
Epoch 2/5
240/240 [==============================] - 55s 228ms/step - loss: 0.4375 - accuracy: 0.8986 - val_loss: 0.3837 - val_accuracy: 0.9161
Epoch 3/5
240/240 [==============================] - 56s 231ms/step - loss: 0.3842 - accuracy: 0.9158 - val_loss: 0.3388 - val_accuracy: 0.9269
Epoch 4/5
240/240 [==============================] - 55s 229ms/step - loss: 0.3587 - accuracy: 0.9248 - val_loss: 0.3382 - val_accuracy: 0.9319
Epoch 5/5
240/240 [==============================] - 55s 229ms/step - loss: 0.3297 - accuracy: 0.9329 - val_loss: 0.2885 - val_accuracy: 0.9421


In [ ]:
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
#calculating epsilon (privacy budget) for our model parameters
epsilon,_=tensorflow_privacy.compute_dp_sgd_privacy(n=train_data.shape[0],
                                              batch_size=batch_size,
                                              noise_multiplier=noise_multiplier,
                                              epochs=epochs,
                                              delta=1e-5)

The epsilon for the differential training comes out to be 0.6353049965480387


In [ ]:
# we can also calculate
tensorflow_privacy.compute_dp_sgd_privacy_statement(number_of_examples=train_data.shape[0],
                                              batch_size=batch_size,
                                              noise_multiplier=noise_multiplier,
                                              used_microbatching=250,
                                              num_epochs=epochs,
                                              delta=1e-5)

'DP-SGD performed over 60000 examples with 250 examples per iteration, noise\nmultiplier 1.3 for 5 epochs with microbatching, and no bound on number of\nexamples per user.\n\nThis privacy guarantee protects the release of all model checkpoints in addition\nto the final model.\n\nExample-level DP with add-or-remove-one adjacency at delta = 1e-05 computed with\nRDP accounting:\n    Epsilon with each example occurring once per epoch:        21.254\n    Epsilon assuming Poisson sampling (*):                      3.782\n\nNo user-level privacy guarantee is possible without a bound on the number of\nexamples per user.\n\n(*) Poisson sampling is not usually done in training pipelines, but assuming\nthat the data was randomly shuffled, it is believed the actual epsilon should be\ncloser to this value than the conservative assumption of an arbitrary data\norder.\n'

**The epsilon for the differential training comes out to be a total of 3.782**


60000 examples with 250 examples per iteration: This indicates that the training dataset consists of 60,000 examples, and during each training iteration, 250 examples are used.

Noise multiplier 1.3 for 5 epochs with microbatching: The noise multiplier represents the amount of noise added to the gradient updates to ensure privacy. This value of 1.3 indicates the level of privacy protection. The training is performed for 5 epochs, and microbatching is a technique used to aggregate gradients in smaller batches to improve privacy.

Privacy guarantee: The privacy guarantee here protects both the release of model checkpoints during training and the final trained model.

Example-level DP with add-or-remove-one adjacency at delta = 1e-05: This appears to be a privacy analysis result using Renyi Differential Privacy (RDP) accounting. It provides an estimate of the privacy level achieved during training.

Epsilon with each example occurring once per epoch: This value (21.254) represents the privacy budget (epsilon) when considering each example in the dataset occurring once in each training epoch. It quantifies the overall privacy protection level during training.

Epsilon assuming Poisson sampling: This value (3.782) is a more optimistic estimate of the privacy level, assuming that the data was randomly shuffled (Poisson sampling). It suggests that if the data order is not arbitrary and follows some randomness, the actual privacy level could be closer to this value.